# Exercise 13: Different linkage, different hierarchical clustering!

In the video, you saw a hierarchical clustering of the voting countries at the Eurovision song contest using `'complete'` linkage. Now, perform a hierarchical clustering of the voting countries with `'single'` linkage, and compare the resulting dendrogram with the one in the video.  Different linkage, different hierarchical clustering!

First, we need to do a little pre-processing to account for one of the Eurovision rules: countries are not allowed to vote for themselves.

**Step 1:** Load the DataFrame _(written for you)_

In [1]:
import pandas as pd

scores_df = pd.read_csv('../datasets/eurovision-2016-televoting.csv', index_col=0)
country_names = list(scores_df.index)

**Step 2:** Display the DataFrame, and have a look.  Each row represents a country that _voted_, while each column represents a country that _performed_.

Notice the NaN ("not-a-number") values.  These correspond to missing scores in the original CSV file.  These scores are missing because countries that performed were not allowed to vote for themselves.

In [17]:
scores_df.sample(10)

,Armenia,Australia,Austria,Azerbaijan,Belgium,Bulgaria,Croatia,Cyprus,Czech Republic,France,...,Lithuania,Malta,Poland,Russia,Serbia,Spain,Sweden,The Netherlands,Ukraine,United Kingdom
From country,,,,,,,,,,,,,,,,,,,,,
United Kingdom,0.0,6.0,0.0,0.0,0.0,8.0,0.0,2.0,0.0,0.0,...,12.0,0.0,10.0,7.0,0.0,4.0,1.0,0.0,5.0,NaN
Malta,0.0,12.0,0.0,6.0,0.0,8.0,0.0,0.0,0.0,2.0,...,0.0,NaN,0.0,10.0,0.0,0.0,0.0,3.0,4.0,1.0
Israel,6.0,5.0,3.0,2.0,1.0,7.0,0.0,0.0,0.0,12.0,...,0.0,0.0,4.0,10.0,0.0,0.0,0.0,0.0,8.0,0.0
Norway,0.0,8.0,0.0,0.0,2.0,5.0,0.0,0.0,0.0,1.0,...,12.0,0.0,10.0,6.0,0.0,0.0,7.0,0.0,4.0,0.0
Iceland,0.0,8.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,5.0,...,4.0,0.0,10.0,7.0,0.0,0.0,12.0,6.0,0.0,0.0
Finland,0.0,7.0,6.0,0.0,0.0,4.0,0.0,1.0,0.0,3.0,...,0.0,0.0,5.0,8.0,0.0,0.0,10.0,0.0,12.0,0.0
Albania,2.0,12.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,...,4.0,0.0,5.0,7.0,0.0,0.0,3.0,0.0,6.0,0.0
Croatia,0.0,5.0,6.0,0.0,0.0,1.0,NaN,0.0,0.0,2.0,...,0.0,0.0,4.0,8.0,12.0,0.0,7.0,0.0,10.0,0.0
Ireland,0.0,6.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,12.0,0.0,10.0,8.0,0.0,0.0,2.0,0.0,4.0,3.0


In [ ]:
df

**Step 3:** Fill in the NaNs with the highest possible score (12) - we are assuming that countries would vote for themselves, if they had been allowed to do so.  _(This bit written for you)._

In [32]:
new_scores_df = scores_df.fillna(12)

**Step 4:** Import the `normalize` function from `sklearn.preprocessing`.

In [27]:
from sklearn.preprocessing import normalize

In [33]:
new_scores_df.head(5)

,Armenia,Australia,Austria,Azerbaijan,Belgium,Bulgaria,Croatia,Cyprus,Czech Republic,France,...,Lithuania,Malta,Poland,Russia,Serbia,Spain,Sweden,The Netherlands,Ukraine,United Kingdom
From country,,,,,,,,,,,,,,,,,,,,,
Albania,2.0,12.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,...,4.0,0.0,5.0,7.0,0.0,0.0,3.0,0.0,6.0,0.0
Armenia,12.0,0.0,4.0,0.0,0.0,0.0,0.0,6.0,0.0,7.0,...,0.0,5.0,1.0,12.0,0.0,0.0,2.0,0.0,10.0,0.0
Australia,0.0,12.0,3.0,0.0,12.0,10.0,0.0,0.0,0.0,7.0,...,1.0,6.0,0.0,5.0,0.0,2.0,0.0,0.0,8.0,4.0
Austria,0.0,3.0,12.0,0.0,0.0,5.0,0.0,0.0,0.0,1.0,...,0.0,0.0,12.0,8.0,4.0,0.0,7.0,6.0,10.0,0.0
Azerbaijan,0.0,2.0,0.0,12.0,0.0,8.0,0.0,0.0,0.0,4.0,...,0.0,5.0,3.0,12.0,0.0,0.0,0.0,0.0,10.0,0.0


**Step 5:** Apply the normalize function to `scores_df.values`, assigning the result to `samples`.

(Why do we need to normalize?  Because now that the missing values are filled with 12 points, some countries (those that performed) given a greater total number of points when voting.  The `normalize` function corrects for this.) 

In [34]:
samples = normalize(new_scores_df)

In [36]:
samples

array([[0.09449112, 0.56694671, 0.        , ..., 0.        , 0.28347335,
        0.        ],
       [0.49319696, 0.        , 0.16439899, ..., 0.        , 0.41099747,
        0.        ],
       [0.        , 0.49319696, 0.12329924, ..., 0.        , 0.32879797,
        0.16439899],
       ...,
       [0.32879797, 0.20549873, 0.24659848, ..., 0.49319696, 0.28769823,
        0.        ],
       [0.28769823, 0.16439899, 0.        , ..., 0.        , 0.49319696,
        0.        ],
       [0.        , 0.24659848, 0.        , ..., 0.        , 0.20549873,
        0.49319696]])

**Step 6:** Import:
 + `linkage` and `dendrogram` from `scipy.cluster.hierarchy`.
 + `matplotlib.pyplot` as `plt`.

In [ ]:
from scipy.clus

**Step 7:** Perform hierarchical clustering on `samples` using the `linkage()` function with the `method='single'` keyword argument. Assign the result to `mergings`.

**Step 8:** Plot a dendrogram of the hierarchical clustering, using the list `country_names` as the `labels`. In addition, specify the `leaf_rotation=90`, and `leaf_font_size=6` keyword arguments as you have done earlier.

**Step 9:** Compare your dendrogram above to the one in the slides and notice that different linkage functions give different hierarchical clusterings.

Both the linkage functions we've considered, "complete" and "single", have advantages and disadvantages.  In practice, just try both out, and see which dendrogram seems more sensible.